# 👔🤝🤖 **Relationship Manager Agent for Busy Executives**

Executives regularly interact with clients, colleagues, investors, and partners—but maintaining these relationships consistently is challenging. Birthdays are forgotten, follow-ups are delayed during busy weeks, and personal touchpoints like greetings or small gifts often fall through the cracks.

This notebook implements a **Relationship Manager Agent**, an AI-powered personal CRM assistant built using the Google Agent Development Kit (ADK).
The agent automates the operational tasks involved in maintaining professional relationships by combining:

* **One main LLM-powered agent** that orchestrates decisions
* **Two sub-agents** (Database Agent + Gift Dispatch Agent)
* **Six custom tools**, including date/time lookup, message scheduling, SQL execution, and gift dispatch
* **Sessions and Memory** for preserving long-term context
* **A2A protocol** for inter-agent communication
* **Observability** through logging and structured outputs

The system follows a modular, multi-agent architecture:

* The **main agent** interprets user instructions, plans actions, and delegates tasks.
* The **Database Agent** handles all persistent operations—adding contacts, describing tables, executing SQL queries.
* The **Gift Dispatch Agent** acts as a simulated third-party vendor that receives structured requests for gift delivery.
* Custom tools enable precise retrieval of date/time, scheduling birthday messages, and dispatching gifts.

```mermaid
flowchart LR
    U["Executive User"] -- Natural language requests --> A["Relationship Manager Agent (LLM-powered)"]
    A -- get today's date/time/place --> T1["GetDateTimeAndPlace Tool"]
    A -- schedule birthday greetings --> T2["ScheduleMessage Tool (Birthday Message Scheduler)"]
    A -- store/retrieve contacts and manage meetups --> DB["Database Agent<br>(Sub-agent)"]
    A -- A2A: dispatch gift --> GD["Gift Dispatch Agent (A2A)"]
    DB --> DB_T1["list_tables Tool"] & DB_T2["describe_table Tool"] & DB_T3["execute_query Tool"]
    GD --> GD_T1["schedule_dispatch Tool"]
    A -- read/write --> M["Memory Store (Long-term)"]
    A -- maintain --> S["Session State (Short-term)"]
    A -- Final structured response --> U

     U:::user
     A:::agent
     T1:::tool
     T2:::tool
     DB:::subagent
     GD:::subagent
     DB_T1:::tool
     DB_T2:::tool
     DB_T3:::tool
     GD_T1:::tool
     M:::storage
     S:::storage
    classDef agent fill:#D8E6FF,stroke:#4A75FF,stroke-width:2px,color:#000
    classDef subagent fill:#EAE3FF,stroke:#8A4DFF,stroke-width:2px,color:#000
    classDef tool fill:#E5FFE5,stroke:#3FA43F,stroke-width:2px,color:#000
    classDef storage fill:#FFF2CC,stroke:#C9A102,stroke-width:2px,color:#000
    classDef user fill:#FFE4E4,stroke:#D94A4A,stroke-width:2px,color:#000
```

![architecture-diagram](diagrams/architecture-diagram.png)

This notebook demonstrates how agents can collaborate, invoke tools, maintain state, and deliver reliable outcomes in a real-world workflow. The overall goal is to show how intelligent agents can support executives in keeping relationships warm, timely, and personal—without adding to their cognitive load.

This document walks through:

1. **Environment setup and configuration**
2. **Tool definitions**
3. **Sub-agent creation**
4. **Main agent logic and orchestration**
5. **A2A workflow between agents**
6. **Session creation and agent interaction**
7. **End-to-end demonstration of the relationship management workflow**

By the end of this notebook, we will see a complete multi-agent system that remembers birthdays, tracks meetups, drafts messages, dispatches gifts, and maintains structured data—showcasing the core capabilities of ADK taught in the 5-Day AI Agents Intensive Course.

### 🔐 Initialize Gemini API Key  
This cell loads the Google API key from Kaggle Secrets and configures the environment so the agent can authenticate with Gemini. It ensures the agent has access to the model before any further steps.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### 🧹 Optional Cleanup
Utility commands for clearing working directories or removing temporary files during debugging.

In [2]:
!rm -rf /kaggle/working/*

### 📦 Import Core Python Libraries  
Essential imports used throughout the notebook for JSON handling, HTTP calls, process management, event streaming, and unique ID generation.

In [3]:
import json
import requests
import subprocess
import time
import uuid
import warnings
import sqlite3

from datetime import datetime
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent, LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.models.google_llm import Gemini
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.apps.app import App, ResumabilityConfig, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from zoneinfo import ZoneInfo

print("✅ Required components imported successfully.")

✅ Required components imported successfully.


### 🤖 Select Gemini Model  
Defines the Gemini model variant used by the main agent and sub-agents. This keeps the model reference consistent across the entire application.

In [4]:
MODEL = "gemini-2.0-flash"

### 🔁 Configure Retry Logic for API Calls  
Sets up a retry policy to handle transient errors when communicating with Gemini or during agent tool execution. Ensures stability in long-running agent flows.

In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

### 🧩 Initialize SQLite Magic for Inline Queries  
This cell enables the `%sql` and `%load_ext sql` Jupyter magic commands, allowing SQL queries to run directly inside notebook cells.  
It also initializes a lightweight SQLite database (`sample.db`) that will store contacts, events, and other session data used by the agent.


In [6]:
%load_ext sql
%sql sqlite:///sample.db

## 🗄️ Database Agent — Architecture, Purpose & Tools

This section defines the **Database Agent**, a dedicated sub-agent responsible for all 
database interactions within the Relationship Manager Agent ecosystem.

In alignment with ADK best practices, the Database Agent encapsulates all low-level 
persistence logic so the main agent remains focused on reasoning, workflow orchestration, 
and high-level decision-making.

The Database Agent exposes three structured tools:

 • `list_tables` \
 • `describe_table` \
 • `execute_query` 

Executes SELECT, INSERT, UPDATE, and DELETE operations using safe, structured inputs.  
This is the core tool that powers contact storage, meetup retrieval, and other CRUD operations.

### 🧩 Why a Separate Database Agent?

- Follows the **single-responsibility principle**  
- Keeps storage concerns isolated from LLM reasoning  
- Makes the system modular and easier to debug  
- Demonstrates the ADK concept of **multi-agent collaboration**  

### 🤝 How It Fits Into the System

The main Relationship Manager Agent delegates all persistent storage tasks—such as adding contacts, retrieving birthdays, and updating meetups—to the Database Agent.

```mermaid
sequenceDiagram
    autonumber

    participant U as Executive User
    participant A as Main Relationship<br/>Manager Agent
    participant DB as Database Agent
    participant LT as list_tables Tool
    participant DT as describe_table Tool
    participant EQ as execute_query Tool

    U->>A: "Add John Doe to my contacts"
    A->>A: Parse instruction & create contact payload

    A->>DB: request (INSERT contact)
    DB->>EQ: execute_query (INSERT INTO people ... )
    EQ-->>DB: Query result (success)

    DB-->>A: response (contact added)
    A-->>U: “John Doe has been added to your contacts.”
```

![database-agent-seq_diag](diagrams/database-agent-seq_diag.png)

All subsequent code cells in this section collectively implement the Database Agent, its 
three tools, and the connection logic required for the Relationship Manager Agent to work with 
persistent data.

### 🗃️ Create and Populate SQLite Tables  
This cell initializes the database schema used by the Database Agent.  
Two tables are created:

**1. `people` table**  
Stores contact information, including names, addresses, and birthday details.

**2. `meetups` table**  
Tracks upcoming or past meetups linked to a specific person through a foreign key.

After creating the tables, sample data is inserted into both tables.  
This provides a realistic starting dataset for the agent to query, schedule meetups, and generate insights during the demo.


In [7]:
%%sql

DROP TABLE IF EXISTS people;
DROP TABLE IF EXISTS meetups;

-- Create the 'people' table
CREATE TABLE IF NOT EXISTS people (
  	person_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255),
    address VARCHAR(1023),
  	birthday_day INTEGER,
    birthday_month INTEGER
  );

-- Create the 'meetups' table
CREATE TABLE IF NOT EXISTS meetups (
  	meetup_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	date_time DATETIME NOT NULL,
    place VARCHAR(1023),
    person_id INTEGER NOT NULL,
  	FOREIGN KEY (person_id) REFERENCES people (person_id)
);

-- Insert data into the 'people' table
INSERT INTO people (first_name, last_name, address, birthday_day, birthday_month) VALUES
('Aarav', 'Sharma', '54 MG Road, Bengaluru', 12, 3),
('Meera', 'Kapoor', '92 Palm Residency, Mumbai', 27, 11),
('Karan', 'Talwar', '18 Sector 22, Chandigarh', 4, 7),
('Rohan', 'Singh', '221B Lodhi Colony, New Delhi', 15, 1),
('Tina', 'Mehta', '44 Queens Street, Kolkata', 30, 10),
('Elena', 'Fernandes', '77 Sunset Boulevard, Goa', 9, 2),
('Vikram', 'Desai', '101 Harmony Towers, Pune', 2, 12),
('Priya', 'Rathod', '66 Marine Heights, Mumbai', 23, 5),
('Daniel', 'Thomas', '12 Green Park, Delhi', 14, 8),
('Sophia', 'Chopra', '33 Hill Crest, Dehradun', 19, 6);

-- Insert data into the 'meetups' table
INSERT INTO meetups (date_time, place, person_id) VALUES
('2025-12-05 10:30:00', 'Blue Tokai Cafe, Bengaluru', 1),
('2025-12-12 19:00:00', 'Olive Bar & Kitchen, Mumbai', 5),
('2026-01-20 15:00:00', 'The Lalit, Chandigarh', 3),
('2025-01-07 13:00:00', 'Cafe Delhi Heights, New Delhi', 7),
('2025-01-18 11:00:00', 'Roastery Coffee House, Kolkata', 4);

 * sqlite:///sample.db
Done.
Done.
Done.
Done.
10 rows affected.
5 rows affected.


[]

### 🔗 SQLite Connection Setup  
Initializes the connection to the SQLite database, ensuring tables exist.  
Used by the Database agent during `execute_query` execution.


In [8]:
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

### 📋 list_tables Tool  
Returns the list of database tables.  
Allows the main agent to inspect or verify database structure programmatically.


In [9]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['people', 'sqlite_sequence', 'meetups']

### 🧩 describe_table Tool  
Provides schema metadata for a given table, including column names and types.  
Useful for dynamic reasoning and validation.

In [10]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("people")
describe_table("meetups")

 - DB CALL: describe_table(people)
 - DB CALL: describe_table(meetups)


[('meetup_id', 'INTEGER'),
 ('date_time', 'DATETIME'),
 ('place', 'VARCHAR(1023)'),
 ('person_id', 'INTEGER')]

### 📝 execute_query Tool  
Executes structured SQL queries for inserting, updating, deleting, or retrieving contact/event data.  
This tool performs the core database operations behind the relationship manager.

In [11]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from meetups")

 - DB CALL: execute_query(select * from meetups)


[(1, '2025-12-05 10:30:00', 'Blue Tokai Cafe, Bengaluru', 1),
 (2, '2025-12-12 19:00:00', 'Olive Bar & Kitchen, Mumbai', 5),
 (3, '2026-01-20 15:00:00', 'The Lalit, Chandigarh', 3),
 (4, '2025-01-07 13:00:00', 'Cafe Delhi Heights, New Delhi', 7),
 (5, '2025-01-18 11:00:00', 'Roastery Coffee House, Kolkata', 4)]

### Define the Database Agent  
Finally, define the agent dedicated to database interactions.  
The agent encapsulates all storage-related operations to keep the main relationship manager agent clean and modular.


In [12]:
db_tools = [list_tables, describe_table, execute_query]

database_agent = LlmAgent(
    name="database_agent",
    model=Gemini(
        model=MODEL,
        retry_options=retry_config
    ),
    description="An agent that can retrieve and update people and meetups data in the database",
    instruction="""
    You are a helpful chatbot that can interact with an SQL database
    which contains information about people. You can use list_tables to see the list
    of all tables, describe_table to understand the table schema of any table,
    and execute_query to retreive or insert data into the db.
    You will take the users questions and compose a suitable SQL
    queries using the tools available to find information required to answer those questions.
    Once you have the information you need, you will
    answer the user's question using the data returned. Search throroughly through all data
    in case you have difficulty finding something.
    """,
    tools=db_tools
)

print("✅ Database Agent defined.")

✅ Database Agent defined.


## 🎁 Gift Dispatch Agent — Remote Service Simulation via A2A

This section defines the **Gift Dispatch Agent**, a remote sub-agent designed to simulate a 
remote third-party gifting service. Executives often rely on external vendors for 
sending birthday gifts, corporate merchandise, or personalized items.  
This agent models that real-world workflow through the **A2A (Agent-to-Agent) protocol**.

The Gift Dispatch Agent exposes a single tool internally:

### • `schedule_dispatch` tool
Accepts a structured payload containing recipient details and gift metadata, then 
returns a mock dispatch confirmation.  
It simulates how a real gifting API might behave, including validation and status reporting.

### 🎯 Why a Separate Gift Dispatch Agent?

- Represents a **remote external vendor**, not internal logic  
- Demonstrates cross-agent communication using **A2A**  
- Keeps specialized operations outside the main agent  
- Shows a realistic use-case of delegating tasks to service-specific sub-agents  
- Enhances modularity and clarity in the architecture

### 🤝 How It Fits Into the System

The main Relationship Manager Agent sends a structured gift-dispatch request to this 
sub-agent whenever the user wants to send a birthday gift or personalized package.  
The Gift Dispatch Agent validates the request and returns a deterministic confirmation 
(ideal for traceability and evaluation).

```mermaid
sequenceDiagram
    autonumber

    participant U as Executive User
    participant A as Main Relationship Manager Agent
    participant GD as Gift Dispatch Agent (A2A)
    participant T as schedule_dispatch Tool

    U->>A: "Send a birthday gift to John Doe"
    A->>A: Parse request & collect gift details
    A->>GD: A2A request with structured payload (recipient, address, gift item, etc.)
    GD->>T: Execute schedule_dispatch with provided payload
    T-->>GD: Dispatch confirmation (dispatch_id, status)
    GD-->>A: A2A response with dispatch result
    A-->>U: “Gift scheduled successfully!”
```

![gift-dispatch-agent-seq_diag](diagrams/gift-dispatch-agent-seq_diag.png)

### The next cells implement:
- the `schedule_dispatch` tool
- the Gift Dispatch Agent definition
- the server process that runs the gift dispatch agent.
- the A2A integration with the main agent

#### Define Gift Dispatch Agent & A2A Service Code

In [13]:
gift_dispatch_service_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def schedule_dispatch(sender_name: str, recipient_name: str, address: str, delivery_date: str, item: str) -> str:
    """Schedule gift dispatches to be sent to specific people.

    This tool simulates scheduling gift dispatches to be sent to the person provided by the user.

    Args:
        sender_name: Name of the sender
        recipient_name: Name of the recipient
        address: recipient's address
        delivery_date: date on which the gift is targeted to reach the recipient
        item: name of the gift item

    Returns:
        String that holds the confirmation message for the dispatch.
        
    """

    print('Gift dispatch successfully scheduled!');
    print(f"Recipient: {recipient_name}")
    print(f"Address: {address}")
    print(f"Delivery date: {delivery_date}")
    print(f"Gift: {item}")
    print("-"*10)

    return f"Your {item} gift has been scheduled successfully to {recipient_name} at Address: {address} on {delivery_date}!"
    # return "Gift has been scheduled successfully!";
    
# Gift dispatch service

gift_dispatch_service_agent = LlmAgent(
    name="gift_dispatch_service_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="External vendor's gift dispatch service agent",
    instruction="""
    You are a gift dispatch service agent. Whenever you are asked to schedule a gift dispatch
    gather all the required details and use schedule_dispatch to scheudle the dispatch. If any
    of the details are missing ask for it. After it's done, send a confirmation message returned
    by the schedule_dispatch tool to the user.
    """,
    tools=[schedule_dispatch],
)


app = to_a2a(gift_dispatch_service_agent, port=8001) 

'''

# Write the gift dispatch agent to a temporary file
with open("/tmp/gift_dispatch_server.py", "w") as f:
    f.write(gift_dispatch_service_agent_code)

print("📝 Gift dispatch service agent code saved to /tmp/gift_dispatch_server.py")

📝 Gift dispatch service agent code saved to /tmp/gift_dispatch_server.py


#### Launch Gift Dispatch Agent as an A2A Microservice

This cell starts the Gift Dispatch Agent as a standalone **HTTP microservice** using
`uvicorn`. The service exposes the agent’s A2A interface so the main Relationship Manager Agent
can communicate with it just like a remote third-party vendor.

##### 🔌 1. Start the Server in the Background
`subprocess.Popen` launches:
- the `gift_dispatch_server:app` module  
- on `localhost:8001`  
- with environment variables forwarded (including `GOOGLE_API_KEY`)  
Standard output is suppressed to keep the notebook clean.


In [14]:
# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "gift_dispatch_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting Gift Dispatcher Agent server...")
print("   Waiting for server to be ready...")

🚀 Starting Gift Dispatcher Agent server...
   Waiting for server to be ready...


##### ⏳ 2. Wait for the Service to Become Ready  
The loop polls `.well-known/agent-card.json` for up to 30 attempts.  
This file is automatically generated by ADK and indicates that the A2A agent service
is active and healthy.

Once detected, the notebook prints:
- the service URL  
- the location of the agent card  

If the service does not start in time, a warning is shown.


In [15]:
# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Gift Dispatch Service Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

...
✅ Gift Dispatch Service Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


##### 📌 3. Store Process Reference  
The server process is stored in a global variable so it can be terminated later if needed.

This setup allows the Gift Dispatch Agent to run in parallel with the main notebook and
respond to A2A requests during the relationship management workflow.

In [16]:
# Store the process so we can stop it later
globals()["gift_dispatch_server_process"] = server_process

##### 📡 Verify Gift Dispatch Agent Server & Fetch Agent Card

This cell checks whether the Gift Dispatch Agent (running on port 8001) is active by 
querying its `.well-known/agent-card.json` endpoint — an automatic metadata file generated 
for all A2A-enabled agents.

If the server is running, the cell:

- Retrieves the full **agent card**  
- Prints structured metadata including name, description, URL, and exposed skills/tools  
- Confirms that the gift dispatch microservice is ready for interaction  

If the service is unreachable or returns an error, a clear diagnostic message is printed 
to help the user verify that the A2A agent server started correctly.


In [17]:
# Start the server process for remote gift dispatcher agent.

try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Gift Dispatch Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Gift Dispatch Agent server is running.")

📋 Gift Dispatch Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor's gift dispatch service agent",
  "name": "gift_dispatch_service_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor's gift dispatch service agent \n    I am a gift dispatch service agent. Whenever I am asked to schedule a gift dispatch\n    gather all the required details and use schedule_dispatch to scheudle the dispatch. If any\n    of the details are missing ask for it. After it's done, send a confirmation message returned\n    by the schedule_dispatch tool to the user.\n    ",
      "id": "gift_dispatch_service_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Schedule gift dispatches to be sent to specific people.\n\nThis tool simulates scheduling gift dispatches to be se

##### 🔗 Create RemoteA2aAgent Proxy for the Gift Dispatch Service

This cell creates a **RemoteA2aAgent**, which acts as a *client-side proxy* for the
Gift Dispatch microservice running at `localhost:8001`.

Instead of calling the HTTP service manually, the main Relationship Manager Agent can now
interact with the Gift Dispatch Agent as if it were a local in-process agent.

##### What this proxy does:
- Reads the agent’s metadata from its `.well-known/agent-card.json` file  
- Understands the agent’s tools, schema, and capabilities  
- Automatically formats A2A requests to the remote service  
- Provides a seamless interface for gift dispatch operations

This RemoteA2aAgent is what enables true **A2A communication** between your main agent
and the external “vendor-like” Gift Dispatch Agent.

Once created, the proxy is ready for use in the main agent workflows.


In [18]:
# Create a RemoteA2aAgent that connects to our Gift dispatcher Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_gift_dispatch_agent = RemoteA2aAgent(
    name="gift_dispatch_service_agent",
    description="Remote gift dispatch service agent from external vendor that schedules gifts to be dispatched.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Gift Dispatch Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")

✅ Remote Gift Dispatch Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


/tmp/ipykernel_47/785720336.py:3: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_gift_dispatch_agent = RemoteA2aAgent(


### 🕒 Custom Tool: GetDateTimeAndPlace  
Defines a simple tool that returns today’s date, time, and the user’s (mocked) location.  
Used by the main agent to make context-aware decisions.


In [19]:
def get_datetime_and_place_for_today():
    return {
        'datetime': datetime.now(tz = ZoneInfo("Asia/Kolkata")).strftime("%Y-%m-%d %H:%M:%S"),
        'place': 'Bangalore, India'
    }

print(get_datetime_and_place_for_today());

{'datetime': '2025-12-01 15:41:46', 'place': 'Bangalore, India'}


### 🎉 Custom Tool: ScheduleMessage  
Implements a mock birthday message scheduler.  
The tool accepts contact details + date and produces a structured event entry.  
This is the primary scheduling mechanism used by the main agent.


In [20]:
# Mock message sender tool

def schedule_message(person_id: int, name: str, message: str):
    """Schedules birthday messages to be sent to specific people.

    This tool simulates scheduling birthday greetings/messages to be sent to the person provided by the user.

    Args:
        person_id: Id of the person who has to be sent the message.
        name: Name of the person
        message: Message to be sent
    """

    print('Message successfully scheduled!');
    print(f"Recipient: {name} / id:{person_id}")
    print(f"Message: {message}")
    print("-"*10)

print(schedule_message('4', 'Suhail', 'A very happy birthday to you!'));

Message successfully scheduled!
Recipient: Suhail / id:4
Message: A very happy birthday to you!
----------
None


## 🧠 Define main Relationship Manager Agent

The next cells construct the **Relationship Manager Agent**, the central LLM-powered orchestrator that acts as a personal assistant for busy executives.  
It ties together the model, retry policy, instructions, tools, and sub-agents to implement the relationship-management workflow.

**Key aspects:**

- **Model & Reliability:**  
  The agent uses the configured `Gemini` model with the established `retry_config` to handle transient API errors reliably.

- **Primary Instruction:**  
  The instruction enforces a critical pre-step: **always call `get_datetime_and_place_for_today`** at the start of any conversation. This ensures every decision is context-aware (local date, time, and place).

- **Responsibility & Capabilities:**  
  The agent:
  - manages contacts and meetups for the executive,
  - schedules messages and gifts,
  - suggests gift/message/place ideas,
  - and interacts with persistent storage and remote vendors via sub-agents.
<br/><br/>
- **Tools Attached:**  
  - `get_datetime_and_place_for_today` — provides contextual date/time/place information.  
  - `schedule_message` — schedules birthday or follow-up messages and returns confirmations.
<br/><br/>
- **Sub-Agents Integrated:**  
  - `database_agent` — handles all DB CRUD operations (via A2A).  
  - `remote_gift_dispatch_agent` — handles gift dispatching as an external vendor (via A2A).
<br/><br/>
- **User Interaction Examples:**  
  The instruction includes example user flows (adding a person, finding birthdays, scheduling meetups, sending messages, dispatching gifts) that guide agent behavior and expected tool/sub-agent usage.

**Why this design:**  
Separating reasoning (main agent) from I/O (database and vendor agents) follows ADK best practices: it keeps the agent focused on high-level decisions while delegating persistent and external operations to specialized sub-agents, enabling modularity, testability, and clear observability.


In [21]:
relationship_manager_agent = Agent(
    name="relationship_manager_agent",
    model=Gemini(
        model=MODEL,
        retry_options=retry_config
    ),
    description="An agent which acts as a personal assistant to a user for managing people relationships.",
    instruction=""""
    Before starting any conversation always call get_datetime_and_place_for_today 
    to know the current date, time and place of the user so that you can answer and
    plan things better.
    
    You are a relationship managing assistant to a user who is a business executive. You help the user
    maintain his professional relationships with people by scheduling gifts, birthday messages, or the
    user's meetups with them on birthdays or whenever the user wants to catch up. 
    
    User can take your help to find gift ideas, message ideas, or place ideas to meet with different people.
    Also, the user can ask you to find information/add information about people and meetups.
    
    You can use database_agent to:
    1. find and add people to the existing database
    2. find upcoming meetups and schedule meetups in the database

    You can call schedule_message to schedule sending messages to specific people that the user wants.
    Always share the confirmation message when you schedule a message.

    You can use remote_gift_dispatch_agent to schedule sending gifts to specific people that the user wants.
    Always share a confirmation message returned by remote_gift_dispatch_agent after dispatching the gift.
    
    Example interactions:
    User > Do you know Elon? If not, add him to the database.
    Assistant > Searches for Elon in the people table and adds an entry if absent.

    User > Find Wexler's birthday.
    Assistant > Searches for Wexler in the people table but could not find an entry.
    So asks the user if Wexler needs to be added in the database. Adds a new entry if user confirms.
    
    User > Book a table with Karan for tomorrow at 7pm at The Lalit's.
    Assistant > Adds a meetup entry in the database for tomorrow with time as 7pm and place as The Lalit's.

    User > Send a birthday greeting to Kim at 10am in the morning.
    Assistant > Searches for Kim and her birthday in the people table.
    Gives suggestions for birthday messages and once confirmed
    uses schedule_message to schedule the birthday greeting message.

    User > Send a wine bottle to Rohit on his birthday.
    Assistant > Searches for Rohit's birthday and address in the people table and
    then uses remote_gift_dispatch_agent to schedule the gift dispatch on Rohit's address. Shows
    a confirmation returned by remote_gift_dispatch_agent.
    """,
    tools=[get_datetime_and_place_for_today, schedule_message],
    sub_agents=[database_agent, remote_gift_dispatch_agent]
)

print("✅ Relationship Manager Agent defined.")

✅ Relationship Manager Agent defined.


### 🧩 Wrap the Relationship Manager Agent into an ADK App

This cell builds the **ADK App** that encapsulates the entire Relationship Manager Agent along with its runtime configuration.  
The `App` object is what the `SessionService` interacts with when running live agent conversations.

**Key components included:**

- **`root_agent=relationship_manager_agent`**  
  Sets the Relationship Manager Agent as the top-level orchestrator for all user interactions.

- **Resumability Configuration**  
  `ResumabilityConfig(is_resumable=True)` enables the app to maintain continuity across turns,  
  allowing the agent to pause, resume, and handle long-running workflows reliably.

- **Logging Plugin**  
  `LoggingPlugin()` records structured logs for every tool call, A2A request, and reasoning step.  
  This improves **observability**, debugging, and transparency in multi-agent workflows.

- **Events Compaction**  
  `EventsCompactionConfig` reduces the size of the conversation event history by:
  - compacting events after every 3 invocations,  
  - keeping 1 turn of overlap for context.  
  This ensures efficient state management without losing continuity.

Together, these settings form the production-ready configuration for deploying or running  
the Relationship Manager Agent in a session-driven environment.


In [22]:
relationship_manager_agent_app = App(
    name="relationship_manager_agent_app",
    root_agent=relationship_manager_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
    # plugins=[LoggingPlugin()], to make the output less verbose for now.
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 5 invocations
        overlap_size=5,  # Keep 10 previous turn for context
    ),
)

/tmp/ipykernel_47/866530039.py:4: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),
/tmp/ipykernel_47/866530039.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### 🏃‍♂️ Initialize Session Service and Runner

This cell sets up the execution environment required for running the Relationship Manager Agent.

#### **1. InMemorySessionService**
Creates an in-memory session store that tracks:
- conversation state  
- event history  
- resumability context  

This enables multi-turn agent interactions without relying on external storage.

#### **2. Runner**
The `Runner` orchestrates all real-time agent activity by connecting:
- the `relationship_manager_agent_app`, and  
- the `session_service`.

It is responsible for:
- executing agent reasoning steps  
- invoking tools  
- forwarding A2A requests  
- managing state transitions  

With both components initialized, the agent is ready to run end-to-end conversations.


In [23]:
session_service = InMemorySessionService()

runner = Runner(
    app=relationship_manager_agent_app,
    session_service=session_service
);

print("✅ Runner created.")

✅ Runner created.


### 💬 Create or Restore Session & Run Interactive Agent Loop

This cell sets up an interactive chat loop that lets you talk directly to the  
Relationship Manager Agent using the ADK runtime.

---

#### **1. Create or Restore Session**
The code attempts to create a new session with:
- a fixed `session_id`
- a test user  

If the session already exists, it gracefully retrieves it.  
This preserves conversation history and allows stateful multi-turn interactions.

---

#### **2. `run_agent()` — Interactive Conversation Function**
A recursive async function that:

- Prompts the user for input (`User >`)  
- Interprets quit commands (`q`, `quit`, `exit`, `goodbye`)  
- Wraps the message into a `types.Content` object  
- Sends it to the agent via `runner.run_async()`  
- Streams back agent messages in real time  
- Prints each generated response (`Agent >`)  
- Calls itself again to continue the dialogue  

This function simulates a live assistant chat, letting you test:
- tool usage  
- A2A interactions  
- DB reads/writes  
- birthday scheduling  
- gift dispatch workflows  
- long-running context and memory

---

#### **3. Launch Conversation**
`await run_agent(False)` starts the assistant loop and keeps the session active  
until the user chooses to exit.

This cell effectively brings the entire multi-agent system to life.

In [26]:
session_id = "session_id_x";

try:
    session = await session_service.create_session(
        app_name=runner.app_name, user_id="test_user", session_id=session_id
    )
except:
    session = await session_service.get_session(
        app_name=runner.app_name, user_id="test_user", session_id=session_id
    )

async def run_agent(exit: bool):
    if(exit):
        return;

    user_input = input("User >");

    if user_input in {"q", "quit", "exit", "goodbye"}:
        print("Thank you! Have a nice day!");
        await run_agent(True);
        return;

    query_content = types.Content(role="user", parts=[types.Part(text=user_input)])

    async for event in runner.run_async(
        user_id="test_user", session_id=session.id, new_message=query_content
    ):
        if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    await run_agent(False);

await run_agent(False);

User > Hi


Agent > Hi! How can I help you manage your relationships today?



User > Find Rohan in our contacts


 - DB CALL: list_tables()


 - DB CALL: describe_table(people)


...

Message successfully scheduled!
Recipient: Rohan Singh / id:1
Message: Dear Rohan, wishing you a very happy birthday! May this year bring you success and happiness.
----------
Agent > Great, I've scheduled the following message to be sent to Rohan Singh: "Dear Rohan, wishing you a very happy birthday! May this year bring you success and happiness."



User > Schedule a gift dispatch for Rohan on Jan 15 2026. Gift should be 'Prada Perfume'. Sender's name is Apar Garg


Agent > I've scheduled the dispatch of your gift to Rohan Singh. It's a Prada Perfume and it will be delivered on January 15, 2026.


User > quit


Thank you! Have a nice day!


## 🔮 Future Work

This notebook implements a complete multi-agent relationship management system using the Google ADK, including custom tools, sub-agents, A2A communication, memory, sessions, and observability.  
Several meaningful enhancements can further extend the functionality and bring the system closer to real-world deployment:

### **1. Real Communication Integrations**
Enable actual message delivery through:
- Gmail or Outlook APIs  
- WhatsApp / SMS gateways  
- Google Calendar integration for real meetups  

This would allow birthday wishes, reminders, and invitations to be sent automatically.

### **2. Production-Grade Gift Dispatching**
Replace the mock gift dispatch service with:
- Amazon Product Ordering APIs  
- Corporate gifting vendors  
- In-house fulfillment systems  

This would turn the A2A gift workflow into a fully actionable feature.

### **3. Smarter Relationship Intelligence**
Extend memory components to track:
- interaction frequency  
- sentiment of past conversations  
- engagement scores  

The agent could then proactively suggest meaningful touchpoints.

### **4. Semantically Advanced Database Queries**
Enhance the Database Agent to support:
- schema-aware reasoning  
- advanced time-based queries  
- grouping people by industry or more useful parameters to plan meetups based on requirements 

This would significantly improve the agent’s analytical abilities.

### **5. Deployment as a Persistent Service**
Run the system continuously using:
- Agent Engine  
- Cloud Run  
- Cloud Functions  

This would support push notifications and proactive reminders.

### **6. Multi-Modal Enhancements**
Integrate:
- voice input/output  
- image-based memory for business cards  
- map-based meetup suggestions  

These additions would enrich the user’s interaction experience.

---

This roadmap highlights how the current prototype can evolve into a full-fledged executive relationship intelligence assistant.
